#### Libraries

In [1]:
import pickle as pkl
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
from itertools import chain
from scipy import sparse
import scipy.sparse as ss

#### Helper Functions

In [ ]:

# def SortLabels(GF, nodeindex, labeldict):
#     '''Get all neighbours of a node at nodeindex and sort them'''
#     neigh = list(GF.neighbors(nodeindex))
#     label = [labeldict[i] for i in neigh]
#     label = list(chain.from_iterable(label))
#     label.sort()
#     return label

# def GlueLabels(nodelabeldict, index, newlist):
#     '''Glue together a list of nodelabels with the node's own label'''
#     nodelbl = nodelabeldict[index][0]
#     x = ''.join([str(s) for s in newlist])
#     return str(nodelbl) + x

# def dictToArray(dict):
#     '''Helper function'''
#     a = np.array(list(dict.items()), dtype=object )
#     a[:,1] = list(chain.from_iterable(a[:,1]))
#     return a

# def draw(G):
#     '''Draw graph with its labels'''
#     pos = nx.spring_layout(G, seed=1, k=0.3)
#     nx.draw(G, pos)
#     node_labels = nx.get_node_attributes(G, "labels")
#     nx.draw_networkx_labels(G, pos, node_labels, font_size=8);

# def getnodelblarr(GF):
#     '''For a specific graph, generate a (sorted) dataframe of old labels and new glued labels for each node'''
#     #Dictionary of labels for graph
#     NLbl_dict = nx.get_node_attributes(GF, 'labels')
#     #Generate new lbls for each vertex
#     newlabels = []
#     for v in NLbl_dict:
#         sortedlbls = SortLabels(GF, v, NLbl_dict)
#         gluedlbl = GlueLabels(NLbl_dict, v, sortedlbls)
#         newlabels.append(gluedlbl)

#     #Array with columns: node, label, newlabel
#     Nlbl_arra0 = np.c_[dictToArray(NLbl_dict), newlabels]
#     #And sort
#     Nlbl_arra0 = Nlbl_arra0[Nlbl_arra0[:,2].argsort()]
#     return Nlbl_arra0

# def hashtodic(ALPHAbet, newlblarr, currentmax):
#     '''Function to hash newly glued labels; then add the (unique) new ones to the overall alphabet.
#     Return a Dataframe with new, old, hashed labels for each node'''
#     #Get unique entries
#     a = np.unique(newlblarr[:,2]) 
#     #Get those that are new also
#     a = [a[i] for i in range(len(a)) if a[i] not in ALPHAbet]
#     #Hash values
#     b = np.arange(len(a))+currentmax
#     currentmax = currentmax+len(b)
#     dic1 = ALPHAbet | {a[i] : b[i] for i in range(len(a))}
#     #relabel
#     newlblarr = np.c_[newlblarr, [dic1[newlblarr[:,2][_]] for _ in range(len(newlblarr))]]    
    
#     return dic1, pd.DataFrame(newlblarr, columns=["node", 'oldlbl', 'hashed', 'newlbl']), currentmax
 
# def assignewlabels(GF, nlblarr):
#     '''Helper function: change a graph's labels. (Change directly as working on a copy) '''
#     newdict = {nlblarr['node'][i] : [nlblarr['newlbl'][i]] for i in range(len(nlblarr))}
#     H = GF.copy()
#     nx.set_node_attributes(H, newdict, name='labels')
#     return H


# def save_sparse_csr(filename, array):
#     np.savez(filename, data=array.data, indices=array.indices,
#              indptr=array.indptr, shape=array.shape)

# def load_sparse_csr(filename):
#     loader = np.load(filename)
#     return sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
#                       shape=loader['shape'])


#### Main Loop

In [2]:
from lib import getnodelblarr, hashtodic, assignewlabels
def WL(graphs, h=4):
    #Number of hops
    h = 4
    N = len(graphs)# N = len(train_graphs)//1
    
    hashedgraphs = graphs#train_graphs.copy()
    hashedgraphs = hashedgraphs[0:N]
    Alphabet = {}#{f'{i}': i for i in range(49)} #{}
    currentmax = 49

    #Alphabet instances for each graph over all hops
    #List of N empty lists
    l = [[] for _ in range(len(hashedgraphs))]
    for i in range(len(hashedgraphs)):
        a = nx.get_node_attributes(hashedgraphs[i], 'labels').values()
        a = list(chain.from_iterable(a))
        l[i] = l[i] + a
    pd.value_counts(l[0])

    print(f'Hops = {h}')
    maxes = [currentmax]
    #For each hop
    for _ in range(h):
        #For each graph
        for i in tqdm(range(len(hashedgraphs))):
            Gi = hashedgraphs[i].copy()# Gi = train_graphs[i]
            #get biggest connected subgraph
            Gi = Gi.subgraph(sorted(nx.connected_components(Gi), key=len, reverse=True)[0])
            #For each node
            #Array with columns: node, label, newlabel
            lblarri = getnodelblarr(Gi)
            #Update big alphabet, hash
            Alphabet, lblarri, currentmax = hashtodic(Alphabet, lblarri, currentmax=currentmax)
            #sort and reset index
            lblarri = lblarri.sort_values(by='node').reset_index(drop=True)
            # lblarri = lblarri
            #relabel to a different graph
            Gi = assignewlabels(Gi, lblarri)
            #assign graph-value
            hashedgraphs[i] = Gi
            #add counts of labels
            a = nx.get_node_attributes(hashedgraphs[i], 'labels').values()
            a = list(chain.from_iterable(a))
            l[i] = l[i] + a
    return hashedgraphs, Alphabet, l

In [3]:
with open('Data/training_data.pkl', 'rb') as file:
    train_graphs = pkl.load(file)
with open('Data/test_data.pkl', 'rb') as file:
    test_graphs = pkl.load(file)
    
hashedgraphs, Alphabet, l = WL(train_graphs+test_graphs)

Hops = 4


100%|██████████| 8000/8000 [00:52<00:00, 153.78it/s]


#### Generate feature vectors

In [4]:
def generateFeatureVectors(hashedgraphs, Alphabet, l):
    '''Creating matrix of feature vectors for each graph'''
    M = np.zeros((len(hashedgraphs), len(Alphabet)+100))
    for i in tqdm(range(len(hashedgraphs))):
        a = pd.value_counts(l[i])
        M[i, list(a.index)] = a.values

    sM = sparse.csr_matrix(M)
    #And save so this only needs to be saved once.
    return sM

sM = generateFeatureVectors(hashedgraphs, Alphabet, l)

100%|██████████| 8000/8000 [00:02<00:00, 3909.40it/s]


Save Data for easier reloading later in sparse format

In [6]:
from lib import save_sparse_csr, load_sparse_csr
save_sparse_csr('Data/WL_allfeatures.npz', sM)
save_sparse_csr('Data/WL_trainfeatvec.npz', sM[0:len(train_graphs)])
save_sparse_csr('Data/WL_testfeatvec.npz', sM[len(train_graphs):len(train_graphs+test_graphs)])

#### Create kernel matrix

In [7]:
def kernelFromFeatureVectors(filename, sparse=True):
    sM = load_sparse_csr(filename) if sparse else np.loadtxt(filename) #'Data/WLfeaturevectors.npz'
    print(f'sM.shape = {sM.shape}')
    M = ss.csr_matrix.toarray(sM)
    K = np.dot(M, M.T)
    sK = ss.csr_matrix(K)
    return sK, M

sK, M = kernelFromFeatureVectors('Data/WL_allfeatures.npz', sparse=True)   
print('Kernel Matrix Determined')
save_sparse_csr('Data/WLKernel_traintest', sK)

# Compare how much sparse arrays save space
# from sys import getsizeof
# M = ss.csr_matrix.toarray(sM)
# print(M.shape, getsizeof(M), getsizeof(sM), sep='\n')


sM.shape = (8000, 69296)
Kernel Matrix Determined


#### Classify

Load data

In [8]:
from sklearn.model_selection import train_test_split
with open('Data/training_labels.pkl', 'rb') as file:
    labels = pkl.load(file)
from lib import load_sparse_csr
WLData = load_sparse_csr('Data/WLKernel_traintest.npz')
WLData = ss.csr_matrix.toarray(WLData)

Project entire dataset (train-validate-test) to PCA

In [9]:
import numpy as np
evals, evecs = np.linalg.eigh(WLData)
numpca = 6000
WLDataNew = WLData.dot(evecs[0:numpca].T)

Split data

In [ ]:
WLTrainValid = WLDataNew[0:len(train_graphs), 0:len(train_graphs)]
X_train, X_validate, y_train, y_validate = train_test_split(WLTrainValid, labels, test_size=0.3, random_state=1)
X_test = WLDataNew[len(train_graphs): len(train_graphs+test_graphs)]

Train Linear Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
lr = LogisticRegression(random_state=0, solver='newton-cholesky')
# Fit on training data
lr.fit(X_train, y_train)

Predict on Validation Data

In [ ]:
#Predict on validation data
pred = lr.predict(X_validate)
fpr, tpr, thresholds = metrics.roc_curve(y_validate, pred, pos_label=1)
#AUC metric
print(f'AUC: {metrics.auc(fpr, tpr)}')

Predict on testing data

In [ ]:
from lib import calculateLogits
pred_test = lr.predict_proba(X_test)
logit = calculateLogits(pred_test)

Save to relevant file

In [ ]:
from lib import saveDataToFormattedSubmissionFile
ctr = int(input('Current submission attempt:\t'))
saveDataToFormattedSubmissionFile(logit, f'Data/WL_test_pred{ctr}.csv')